# Containers
Module：所有神经网络模块的基类，用于构建自定义模型。

Sequential：按顺序组合多个子模块，适用于线性堆叠的模型。

ModuleList：存储任意数量的子模块，适用于需要动态管理模块的场景。

ModuleDict：以字典形式存储子模块，适用于需要通过键访问模块的场景。

ParameterList：存储一系列参数，适用于需要动态管理参数的场景。

ParameterDict：以字典形式存储参数，适用于需要通过键访问和管理参数的场景。

In [1]:
import torch
import torch.nn as nn

# 自定义的神经网络继承自 nn.Module
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 5)  # 定义一个线性层
        self.fc2 = nn.Linear(5, 2)   # 定义另一个线性层

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # 使用 ReLU 激活函数
        x = self.fc2(x)
        return x

model = SimpleNN()
print(model)

SimpleNN(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=2, bias=True)
)


In [2]:
# 使用 Sequential 构建模型
model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 2)
)
print(model)

Sequential(
  (0): Linear(in_features=10, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=2, bias=True)
)


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(10, 10) for _ in range(3)])  # 三个线性层

    def forward(self, x):
        for layer in self.layers:
            x = torch.relu(layer(x))  # 依次应用每个线性层
        return x

model = Net()
print(model)

Net(
  (layers): ModuleList(
    (0-2): 3 x Linear(in_features=10, out_features=10, bias=True)
  )
)


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.ModuleDict({
            'fc1': nn.Linear(10, 5),
            'fc2': nn.Linear(5, 2)
        })

    def forward(self, x, layer_name):
        x = torch.relu(self.layers[layer_name](x))  # 根据键名选择层
        return x

model = Net()
print(model)

Net(
  (layers): ModuleDict(
    (fc1): Linear(in_features=10, out_features=5, bias=True)
    (fc2): Linear(in_features=5, out_features=2, bias=True)
  )
)


In [5]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(10, 10)) for _ in range(3)])  # 三个参数，parameterList一般放的都是矩阵，与ModuleList不同

    def forward(self, x):
        for param in self.params:
            x = x @ param  # 矩阵乘法
        return x

model = MyModel()
print(model)

MyModel(
  (params): ParameterList(
      (0): Parameter containing: [torch.float32 of size 10x10]
      (1): Parameter containing: [torch.float32 of size 10x10]
      (2): Parameter containing: [torch.float32 of size 10x10]
  )
)


In [6]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.params = nn.ParameterDict({
            'weight1': nn.Parameter(torch.randn(10, 5)),
            'weight2': nn.Parameter(torch.randn(5, 2))
        })

    def forward(self, x, param_name):
        x = x @ self.params[param_name]  # 根据键名选择参数
        return x

model = MyModel()
print(model)

MyModel(
  (params): ParameterDict(
      (weight1): Parameter containing: [torch.FloatTensor of size 10x5]
      (weight2): Parameter containing: [torch.FloatTensor of size 5x2]
  )
)


# 卷积层（Convolution Layers）

In [7]:
# nn.Conv1d
# 1D卷积层应用于一维信号（如时间序列、音频信号）。它对多个输入平面（通道）上的输入信号进行一维卷积操作，提取局部特征。

import torch
import torch.nn as nn

conv1d = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=3)
input_signal = torch.randn(20, 16, 50)  # (batch_size, in_channels, signal_length)
output = conv1d(input_signal) # 这里只改变了通道数，长度由于kernelsize也会减小
print(output.shape)  # 输出形状: (20, 33, 48)

torch.Size([20, 33, 48])


In [8]:
# nn.Conv2d
# 2D卷积层用于处理二维输入（如图像）。它在每个输入通道上应用二维卷积操作，非常适合图像处理。

conv2d = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)
image = torch.randn(10, 3, 32, 32)  # (batch_size, in_channels, height, width)
output = conv2d(image)# 这里只改变了通道数，长度由于kernelsize也会减小
print(output.shape)  # 输出形状: (10, 16, 30, 30)

torch.Size([10, 16, 30, 30])


In [ ]:
# nn.Conv3d
# 3D卷积层用于三维数据（如视频或三维医疗图像），在三个维度上执行卷积操作。
conv3d = nn.Conv3d(in_channels=1, out_channels=8, kernel_size=3)
volume = torch.randn(4, 1, 16, 32, 32)  # (batch_size, in_channels, depth, height, width)
output = conv3d(volume)
print(output.shape)  # 输出形状: (4, 8, 14, 30, 30)

In [10]:
# nn.ConvTranspose1d
# 1D反卷积层（又称转置卷积或反向卷积）用于增大一维信号的尺寸，常用于上采样（例如生成模型中的解卷积）。
conv_transpose1d = nn.ConvTranspose1d(in_channels=16, out_channels=33, kernel_size=3)
input_signal = torch.randn(20, 16, 50)
output = conv_transpose1d(input_signal)# 主要的不同就是原信号的长度增加了
print(output.shape)  # 输出形状: (20, 33, 52) 

# nn.ConvTranspose2d
# 2D反卷积用于二维数据（如图像）的上采样，常用于卷积神经网络（CNN）中的生成模型。

conv_transpose2d = nn.ConvTranspose2d(in_channels=16, out_channels=8, kernel_size=3)
image = torch.randn(10, 16, 32, 32)
output = conv_transpose2d(image)
print(output.shape)  # 输出形状: (10, 8, 34, 34)

# nn.ConvTranspose3d
# 3D反卷积用于三维数据的上采样，适合三维体积数据的生成。

conv_transpose3d = nn.ConvTranspose3d(in_channels=8, out_channels=1, kernel_size=3)
volume = torch.randn(4, 8, 16, 32, 32)
output = conv_transpose3d(volume)
print(output.shape)  # 输出形状: (4, 1, 18, 34, 34)

torch.Size([20, 33, 52])
torch.Size([10, 8, 34, 34])
torch.Size([4, 1, 18, 34, 34])


In [ ]:

lazy_conv2d = nn.LazyConv2d(out_channels=16, kernel_size=3)
image = torch.randn(10, 3, 32, 32)  # (batch_size, in_channels, height, width)
output = lazy_conv2d(image)  # 自动推断 in_channels 为 3
print(output.shape)

In [11]:
# nn.Unfold
# Unfold 从输入张量中提取滑动窗口的局部块，非常适合在卷积操作或图像处理时获取局部区域信息。
import torch
import torch.nn as nn

# 定义一个 4x4 的图像，3 通道
image = torch.arange(3*4*4).view(1, 3, 4, 4).float()
print("原始图像：\n", image)

# Unfold操作：提取 2x2 的局部区域
unfold = nn.Unfold(kernel_size=2)
output = unfold(image)
print("Unfold 结果：\n", output)


原始图像：
 tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]],

         [[32., 33., 34., 35.],
          [36., 37., 38., 39.],
          [40., 41., 42., 43.],
          [44., 45., 46., 47.]]]])
Unfold 结果：
 tensor([[[ 0.,  1.,  2.,  4.,  5.,  6.,  8.,  9., 10.],
         [ 1.,  2.,  3.,  5.,  6.,  7.,  9., 10., 11.],
         [ 4.,  5.,  6.,  8.,  9., 10., 12., 13., 14.],
         [ 5.,  6.,  7.,  9., 10., 11., 13., 14., 15.],
         [16., 17., 18., 20., 21., 22., 24., 25., 26.],
         [17., 18., 19., 21., 22., 23., 25., 26., 27.],
         [20., 21., 22., 24., 25., 26., 28., 29., 30.],
         [21., 22., 23., 25., 26., 27., 29., 30., 31.],
         [32., 33., 34., 36., 37., 38., 40., 41., 42.],
         [33., 34., 35., 37., 38., 39., 41., 42., 43.],
         [36., 37., 38., 40., 

In [12]:
# Fold操作：将 2x2 的局部区域重新组合成原始形状
fold = nn.Fold(output_size=(4, 4), kernel_size=2)
reconstructed_image = fold(output)
print("Fold 重构后的图像：\n", reconstructed_image)

Fold 重构后的图像：
 tensor([[[[  0.,   2.,   4.,   3.],
          [  8.,  20.,  24.,  14.],
          [ 16.,  36.,  40.,  22.],
          [ 12.,  26.,  28.,  15.]],

         [[ 16.,  34.,  36.,  19.],
          [ 40.,  84.,  88.,  46.],
          [ 48., 100., 104.,  54.],
          [ 28.,  58.,  60.,  31.]],

         [[ 32.,  66.,  68.,  35.],
          [ 72., 148., 152.,  78.],
          [ 80., 164., 168.,  86.],
          [ 44.,  90.,  92.,  47.]]]])


# Pooling layers


In [13]:
# Max Pooling (最大池化)
import torch
import torch.nn as nn

# 2D Max Pooling 示例
x = torch.randn(1, 1, 4, 4)  # 输入 1x1x4x4 的图像
pool = nn.MaxPool2d(kernel_size=2, stride=2) # 一般kernel_size和stride相同
output = pool(x)
print(output.shape)  # 输出大小变为 1x1x2x2


torch.Size([1, 1, 2, 2])


In [16]:
# 2D Average Pooling 示例
x = torch.randn(1, 1, 4, 4)
pool = nn.AvgPool2d(kernel_size=2, stride=2)
output = pool(x)
print(output.shape)  # 输出大小也是 1x1x2x2


torch.Size([1, 1, 2, 2])


In [17]:
# 自适应池化允许你指定输出的大小，而不是窗口的大小。它会根据输入尺寸自动计算适合的窗口大小和步幅，确保输出的大小固定。

# nn.AdaptiveMaxPool1d / nn.AdaptiveAvgPool1d：对于一维数据，适应性最大池化和平均池化。
# nn.AdaptiveMaxPool2d / nn.AdaptiveAvgPool2d：对于二维数据，如图像，生成固定大小的输出，无论输入大小如何。
# nn.AdaptiveMaxPool3d / nn.AdaptiveAvgPool3d：适用于三维数据，例如视频。
# 2D Adaptive Max Pooling 示例
x = torch.randn(1, 1, 8, 8)
adaptive_pool = nn.AdaptiveMaxPool2d((4, 4))  # 输出固定大小为 4x4
output = adaptive_pool(x)
print(output.shape)  # 输出大小为 1x1x4x4


torch.Size([1, 1, 4, 4])


In [ ]:
# 分数最大池化是一种特殊的池化方式，它允许窗口大小不是整数，这意味着池化的步幅可以是浮点数。它用于那些希望进行更细致下采样的场景。

# nn.FractionalMaxPool2d：应用于二维图像数据，允许进行分数池化。
# nn.FractionalMaxPool3d：应用于三维数据。
# 2D Fractional Max Pooling 示例
x = torch.randn(1, 1, 8, 8)
fractional_pool = nn.FractionalMaxPool2d(kernel_size=3, output_size=(5, 5))
output = fractional_pool(x)
print(output.shape)  # 输出大小为 1x1x5x5

In [ ]:
# Lp 池化是通过计算输入局部区域的 Lp 范数来进行池化的操作。通常，
# p=2 对应二次范数，它用于聚合局部区域的信息。

# nn.LPPool1d：应用于一维数据的 Lp 池化。
# nn.LPPool2d：应用于二维数据的 Lp 池化。
# nn.LPPool3d：应用于三维数据的 Lp 池化。
# 2D Lp Pooling 示例
x = torch.randn(1, 1, 8, 8)
lp_pool = nn.LPPool2d(norm_type=2, kernel_size=2, stride=2)
output = lp_pool(x)
print(output.shape)  # 输出大小为 1x1x4x4


In [ ]:
# 反池化（Unpooling）是一种与池化相反的操作，它将池化操作的结果还原回原始尺寸。反池化通常用于需要恢复原始尺寸的场景，例如在生成对抗网络（GAN）中。
# 反池化除了需要结果以外，还得需要indices，因为池化操作会丢失位置信息，反池化需要知道每个池化窗口的位置，以便正确地恢复原始尺寸。
# 反池化还原的数据会缺失除了最大值以外的所有信息，因为其余部分都用0填充了，所以反池化一般会和原数据一起使用，拼接起来。
# 2D Max Unpooling 示例
x = torch.randn(1, 1, 4, 4)
maxpool = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
unpool = nn.MaxUnpool2d(kernel_size=2, stride=2)

output, indices = maxpool(x)
reconstructed = unpool(output, indices, output_size=x.size())
print(reconstructed.shape)  # 输出大小为 1x1x4x4

# Padding layers


In [18]:
# 反射填充使用输入边界的镜像来填充。
import torch
import torch.nn as nn

input_tensor = torch.tensor([[[1, 2, 3], [4, 5, 6]]], dtype=torch.float32)  # 1D input
pad_layer = nn.ReflectionPad1d(1)
padded_tensor = pad_layer(input_tensor)
print(padded_tensor)


tensor([[[2., 1., 2., 3., 2.],
         [5., 4., 5., 6., 5.]]])


In [19]:
# 复制填充使用输入边界的值进行复制填充。
pad_layer = nn.ReplicationPad1d(1)
padded_tensor = pad_layer(input_tensor)
print(padded_tensor)

tensor([[[1., 1., 2., 3., 3.],
         [4., 4., 5., 6., 6.]]])


In [20]:
# 零填充会在张量边界处填充零。
pad_layer = nn.ZeroPad2d(1)
padded_tensor = pad_layer(input_tensor)
print(padded_tensor)

tensor([[[0., 0., 0., 0., 0.],
         [0., 1., 2., 3., 0.],
         [0., 4., 5., 6., 0.],
         [0., 0., 0., 0., 0.]]])


In [21]:
# 常数填充会在张量边界处填充指定的常数值。
pad_layer = nn.ConstantPad2d(1, 10)  # 用10填充
padded_tensor = pad_layer(input_tensor)
print(padded_tensor)

tensor([[[10., 10., 10., 10., 10.],
         [10.,  1.,  2.,  3., 10.],
         [10.,  4.,  5.,  6., 10.],
         [10., 10., 10., 10., 10.]]])


In [22]:
# 循环填充（也称为环绕填充）使用输入的边界来循环填充。
pad_layer = nn.CircularPad1d(1)
padded_tensor = pad_layer(input_tensor)
print(padded_tensor)

tensor([[[3., 1., 2., 3., 1.],
         [6., 4., 5., 6., 4.]]])


各类激活函数在神经网络中扮演着不同的角色，根据任务和模型的需求选择合适的激活函数，可以提升模型的性能与训练效率。以下是 PyTorch 中常见激活函数及其特点与适用场景：

### 1. **`nn.ELU`（Exponential Linear Unit）**
- **特点**：ELU 在输入为负时指数衰减，正值则线性输出，避免了 ReLU 的死区问题（ReLU 在负区间输出为零）。
- **适用场景**：在有些任务中，ELU 可能比 ReLU 收敛更快，适合深层神经网络。

### 2. **`nn.Hardshrink`**
- **特点**：Hardshrink 是一种稀疏激活函数，将小于阈值的输入截断为零。
- **适用场景**：适用于需要稀疏输出的场景，比如特征选择或去噪。

### 3. **`nn.Hardsigmoid`**
- **特点**：简化的 Sigmoid 函数，计算量更少。输出值在 `[0, 1]` 之间，具有阶梯状的近似线性区域。
- **适用场景**：适合在资源受限的应用中使用，或者当标准 Sigmoid 过于复杂时作为替代。

### 4. **`nn.Hardtanh`**
- **特点**：Tanh 的一种近似函数，输入在区间 `[-1, 1]` 之间线性输出，其余部分剪裁。
- **适用场景**：和标准 Tanh 类似，适合需要处理带符号输入的任务，同时减少计算复杂度。

### 5. **`nn.Hardswish`**
- **特点**：近似 SiLU 的简化版本，带来与 SiLU 相似的特性，但计算更高效。
- **适用场景**：适用于需要非线性特性的轻量级模型，尤其在移动设备上的应用。

### 6. **`nn.LeakyReLU`**
- **特点**：类似 ReLU，但允许负值以小的斜率通过，避免了 ReLU 死区问题（ReLU 在负值处输出零）。
- **适用场景**：适合深度网络，特别是希望在负值区域保留一些梯度信息的场景。

### 7. **`nn.LogSigmoid`**
- **特点**：对 Sigmoid 进行对数变换，输出的范围是负数，梯度较为平滑。
- **适用场景**：适合对数损失函数或涉及对数计算的场景。

### 8. **`nn.MultiheadAttention`**
- **特点**：实现多头注意力机制，允许模型从不同的表示空间关注输入序列中的不同部分。
- **适用场景**：广泛用于自然语言处理中的 Transformer 模型和序列处理任务。

### 9. **`nn.PReLU`（Parametric ReLU）**
- **特点**：类似于 LeakyReLU，但负斜率是可学习的参数。
- **适用场景**：适合需要在不同层或不同通道中学习不同激活模式的模型。

### 10. **`nn.ReLU`（Rectified Linear Unit）**
- **特点**：正值部分线性输出，负值部分输出为零。非常简单且计算效率高。
- **适用场景**：广泛应用于几乎所有神经网络模型，适合大部分任务，尤其是图像处理任务。

### 11. **`nn.ReLU6`**
- **特点**：ReLU 的一种变体，输出限制在 `[0, 6]` 之间。
- **适用场景**：用于限制输出范围的场景，例如在移动设备中使用的轻量级模型。

### 12. **`nn.RReLU`（Randomized Leaky ReLU）**
- **特点**：LeakyReLU 的随机版本，负斜率在训练时随机选择，在测试时固定。
- **适用场景**：适合需要引入随机性以提高模型泛化能力的场景。

### 13. **`nn.SELU`（Scaled Exponential Linear Unit）**
- **特点**：SELU 是一种带有自归一化特性的激活函数，具有缩放指数衰减的特点。
- **适用场景**：适合深度网络，并且不需要特别处理梯度消失的问题。

### 14. **`nn.CELU`（Continuously Differentiable Exponential Linear Unit）**
- **特点**：ELU 的一种改进版本，具有平滑的微分性。
- **适用场景**：适合需要平滑梯度变化的任务。

### 15. **`nn.GELU`（Gaussian Error Linear Unit）**
- **特点**：使用高斯分布进行激活，具有平滑的非线性。
- **适用场景**：适合 Transformer 等需要复杂非线性激活的网络。

### 16. **`nn.Sigmoid`**
- **特点**：标准的 Sigmoid 函数，输出值在 `[0, 1]` 之间，适合处理概率问题。
- **适用场景**：常用于二分类任务的输出层。

### 17. **`nn.SiLU`（Swish or Sigmoid Linear Unit）**
- **特点**：Swish 激活函数，输出为 `x * sigmoid(x)`，具有平滑的非线性。
- **适用场景**：适合需要精细非线性处理的任务，Google 的研究表明它在某些任务上优于 ReLU。

### 18. **`nn.Mish`**
- **特点**：Mish 函数具有平滑的非线性特性，被认为可以帮助梯度流动并提供更好的性能。
- **适用场景**：在深层网络中可能会提供比 ReLU 更好的表现。

### 19. **`nn.Softplus`**
- **特点**：Softplus 是 ReLU 的平滑版本，输出为正值但没有硬性截断。
- **适用场景**：用于需要平滑激活的场景，避免 ReLU 的硬性截断。

### 20. **`nn.Softshrink`**
- **特点**：Softshrink 在小于阈值时将输入值压缩到零。
- **适用场景**：适合需要稀疏输出的场景，比如去噪或稀疏特征学习。

### 21. **`nn.Softsign`**
- **特点**：将输入缩放到 `[-1, 1]` 区间，梯度变化更平滑，减轻梯度消失问题。
- **适用场景**：适合需要平滑、连续输出的任务。

### 22. **`nn.Tanh`**
- **特点**：双曲正切函数，输出值在 `[-1, 1]` 之间，适合处理带符号的输入。
- **适用场景**：适用于大部分需要对负值敏感的任务。

### 23. **`nn.Tanhshrink`**
- **特点**：将输入减去其 Tanh 输出，使得小值被压缩到零。
- **适用场景**：适合需要一定压缩效果但又不希望完全剪切的任务。

### 24. **`nn.Threshold`**
- **特点**：输入小于阈值时被压缩为一个固定值，大于阈值时保持不变。
- **适用场景**：适合只需要简单门控效果的场景。

### 25. **`nn.GLU`（Gated Linear Unit）**
- **特点**：GLU 使用门控机制，对输入进行线性变换的同时保留一部分信息。
- **适用场景**：用于需要门控机制的任务，如序列处理中的注意力机制。

### 总结
这些激活函数在不同任务和模型结构中发挥不同作用。比如，ReLU 及其变种适合大多数深度学习任务，Sigmoid 和 Softmax 常用于分类任务的输出层，而 SiLU、Mish 等较新的激活函数在一些特定任务上能提供更好的性能。

# 非线性激活函数

In [23]:
# nn.Softmin
# Softmin 将输入张量转换为概率分布，输入越小，输出的概率越大。
import torch
import torch.nn as nn

# 创建一个随机输入张量
x = torch.randn(3)
print("Input:", x)

# 使用 Softmin 激活函数
softmin = nn.Softmin(dim=0)
output = softmin(x)
print("Softmin Output:", output)


# Softmin 函数将输入的每个元素变换为非负数，且这些数的总和为 1，适用于需要把小值对应更大权重的情况。

Input: tensor([-0.4032,  0.3346, -0.1698])
Softmin Output: tensor([0.4405, 0.2106, 0.3488])


In [24]:
# nn.Softmax
# Softmax 是一种常见的激活函数，通常用于分类模型的输出层，将输入转换为概率分布，总和为 1。

# 使用 Softmax 激活函数
softmax = nn.Softmax(dim=0)
output = softmax(x)
print("Softmax Output:", output)


Softmax Output: tensor([0.2297, 0.4803, 0.2900])


In [25]:
# nn.Softmax2d
# Softmax2d 是专门用于 2D 张量（通常是图像）的 Softmax 函数，应用于图像的每个空间位置。
# 创建一个 2D 张量，模拟图像的通道输出
x_2d = torch.randn(2, 3, 4, 4)  # 形状为 (batch_size, channels, height, width)

# 使用 Softmax2d 激活函数
softmax2d = nn.Softmax2d()
output_2d = softmax2d(x_2d)
print("Softmax2d Output:", output_2d)


Softmax2d Output: tensor([[[[0.7692, 0.4978, 0.7621, 0.1117],
          [0.0912, 0.0601, 0.1700, 0.3202],
          [0.7793, 0.0620, 0.1259, 0.5856],
          [0.1780, 0.2955, 0.6483, 0.3088]],

         [[0.1960, 0.2612, 0.1600, 0.1357],
          [0.5358, 0.6786, 0.4128, 0.2732],
          [0.1536, 0.3405, 0.0772, 0.2953],
          [0.6948, 0.3169, 0.1789, 0.4895]],

         [[0.0348, 0.2410, 0.0779, 0.7526],
          [0.3730, 0.2613, 0.4173, 0.4066],
          [0.0671, 0.5975, 0.7970, 0.1192],
          [0.1273, 0.3877, 0.1728, 0.2017]]],


        [[[0.6234, 0.2767, 0.2991, 0.7504],
          [0.2209, 0.4242, 0.0589, 0.3767],
          [0.1096, 0.4359, 0.0438, 0.2790],
          [0.1323, 0.2152, 0.6321, 0.3030]],

         [[0.1173, 0.3958, 0.1302, 0.0986],
          [0.5572, 0.4850, 0.6524, 0.1833],
          [0.4225, 0.3180, 0.3152, 0.0541],
          [0.6448, 0.3654, 0.0531, 0.2471]],

         [[0.2593, 0.3275, 0.5708, 0.1510],
          [0.2219, 0.0908, 0.2887, 0.4400],
  

In [26]:
# nn.LogSoftmax
# LogSoftmax 是 Softmax 函数的对数版本，常用于和交叉熵损失结合，因为它能更好地处理数值稳定性问题。
# 使用 LogSoftmax 激活函数
log_softmax = nn.LogSoftmax(dim=0)
output = log_softmax(x)
print("LogSoftmax Output:", output)

LogSoftmax Output: tensor([-1.4711, -0.7333, -1.2377])


In [55]:
# nn.AdaptiveLogSoftmaxWithLoss
# AdaptiveLogSoftmaxWithLoss 是一种高效的 Softmax 近似方法，适用于大词汇量的多分类任务，可以在保持计算效率的同时减少内存开销。

# 创建一个具有 5 个类别的分类任务，并且假设有 10 个输入特征
in_features = 10
n_classes = 5

# 输入数据和标签
x = torch.randn(3, in_features)  # batch_size=3, input_features=10
target = torch.randint(0, n_classes, (3,))

# 定义 AdaptiveLogSoftmaxWithLoss
adaptive_softmax = nn.AdaptiveLogSoftmaxWithLoss(in_features, n_classes, cutoffs=[2, 4])

# 前向传播
output = adaptive_softmax(x, target)
print("AdaptiveLogSoftmaxWithLoss Output:", output.output)
print("Loss:", output.loss)


AdaptiveLogSoftmaxWithLoss Output: tensor([-1.1073, -1.6581, -1.9083], grad_fn=<AddBackward0>)
Loss: tensor(1.5579, grad_fn=<MeanBackward0>)


在深度学习中，**归一化层**（Normalization Layers）起着至关重要的作用，能够加速训练、提高模型的性能并减少对初始化的敏感性。下面我们将讨论不同类型的归一化层，主要聚焦于它们的不同之处以及适用场景。

### 1. **Batch Normalization (批量归一化)**
- **`nn.BatchNorm1d`**: 应用于 2D 或 3D 输入，常用于处理序列数据或特征数据。
- **`nn.BatchNorm2d`**: 应用于 4D 输入，通常用于卷积神经网络（CNN）的特征图。
- **`nn.BatchNorm3d`**: 应用于 5D 输入，适合处理 3D 卷积（如视频数据）的模型。
- **Lazy Versions**: 
  - **`nn.LazyBatchNorm1d`**: 懒惰初始化的 1D 批量归一化。
  - **`nn.LazyBatchNorm2d`**: 懒惰初始化的 2D 批量归一化。
  - **`nn.LazyBatchNorm3d`**: 懒惰初始化的 3D 批量归一化。

**特点**:
- **Batch Normalization** 是通过对每一批次的数据进行标准化，使其均值为 0，方差为 1。
- 可以缓解梯度消失或爆炸问题，加速收敛。

### 2. **Group Normalization (组归一化)**
- **`nn.GroupNorm`**: 将特征分为多个组，每个组独立进行归一化。与批量归一化相比，它不依赖于批量大小，适用于小批量数据。

**特点**:
- 特别适合小批量（如小型图像分类）或者在分布不均的情况下。
- 可以在不影响性能的前提下，适应不同的网络架构。

### 3. **Instance Normalization (实例归一化)**
- **`nn.InstanceNorm1d`**: 应用于 1D 输入。
- **`nn.InstanceNorm2d`**: 应用于 2D 输入，通常用于图像生成任务。
- **`nn.InstanceNorm3d`**: 应用于 3D 输入。

**特点**:
- 对每一个样本独立进行归一化，非常适合风格迁移、生成对抗网络（GAN）等任务。
- 通过消除样本间的均值和方差的影响，使得模型更关注样本本身的特征。

### 4. **Layer Normalization (层归一化)**
- **`nn.LayerNorm`**: 对每个输入样本的特征进行归一化，通常用于 RNN 等序列模型中。

**特点**:
- 归一化是对每一个样本的所有特征进行处理，与批量归一化的不同之处在于，它不依赖于其他样本的特征。
- 更适合于小批量的场景，因为它不会因批量大小而波动。

### 5. **Local Response Normalization (局部响应归一化)**
- **`nn.LocalResponseNorm`**: 主要用于卷积神经网络中的特征图，通过对邻域的响应进行归一化。

**特点**:
- 增强了模型的局部特征，常用于 AlexNet 等早期 CNN 架构中。

### 6. **RMS Normalization (均方根归一化)**
- **`nn.RMSNorm`**: 一种改进的层归一化方法，使用均方根（RMS）进行归一化。

**特点**:
- 与层归一化类似，但它使用 RMS 而不是均值和方差来进行归一化，可以在一些情况下提高稳定性和效果。

### 总结
- **Batch Normalization** 适合大批量的训练，通常用于 CNN，能有效加速训练。
- **Group Normalization** 是处理小批量的好选择，适合不同的网络结构。
- **Instance Normalization** 在风格迁移和 GAN 中效果显著。
- **Layer Normalization** 适合 RNN 和其他依赖序列的任务。
- **Local Response Normalization** 主要用于早期的 CNN 结构。
- **RMS Normalization** 提供了一种新的归一化方式，适用于特定场景。

选择适合的归一化层可以根据具体的模型结构、任务类型和数据特性来决定。

In [67]:
import torch
import torch.nn as nn

# 输入示例
batch_size = 4
num_features = 4
height = 5
width = 5

# Batch Normalization 示例
# 对于 2D 输入，通常在 CNN 中使用
batch_norm_2d = nn.BatchNorm2d(num_features)
input_2d = torch.randn(batch_size, num_features, height, width)
output_2d = batch_norm_2d(input_2d)

# Group Normalization 示例
group_norm = nn.GroupNorm(num_groups=2, num_channels=num_features)
output_group_norm = group_norm(input_2d)

# Instance Normalization 示例
instance_norm_2d = nn.InstanceNorm2d(num_features)
output_instance_norm = instance_norm_2d(input_2d)

# Layer Normalization 示例
layer_norm = nn.LayerNorm([num_features, height, width])
output_layer_norm = layer_norm(input_2d)  # 保持 3D 输入

# Local Response Normalization 示例
local_response_norm = nn.LocalResponseNorm(size=5)
output_local_response_norm = local_response_norm(input_2d)

# RMS Normalization 示例
# rms_norm = nn.RMSNorm(num_features)
# output_rms_norm = rms_norm(input_2d.view(batch_size, -1))

# 打印输出的形状以验证
print("BatchNorm2d Output Shape:", output_2d.shape)
print("GroupNorm Output Shape:", output_group_norm.shape)
print("InstanceNorm2d Output Shape:", output_instance_norm.shape)
print("LayerNorm Output Shape:", output_layer_norm.shape)
print("LocalResponseNorm Output Shape:", output_local_response_norm.shape)
# print("RMSNorm Output Shape:", output_rms_norm.shape)


BatchNorm2d Output Shape: torch.Size([4, 4, 5, 5])
GroupNorm Output Shape: torch.Size([4, 4, 5, 5])
InstanceNorm2d Output Shape: torch.Size([4, 4, 5, 5])
LayerNorm Output Shape: torch.Size([4, 4, 5, 5])
LocalResponseNorm Output Shape: torch.Size([4, 4, 5, 5])


# Recurrent Layers

RNN层用于简单的序列数据处理。

LSTM层更复杂，能有效处理长期依赖关系。

GRU层比LSTM高效且复杂度低，适合许多任务。

RNNCell、LSTMCell和GRUCell提供对状态管理的精细控制，这对于自定义架构非常有用。

In [71]:
rnn = nn.RNN(input_size=10, hidden_size=20, num_layers=2, nonlinearity='tanh', batch_first=True)

lstm = nn.LSTM(input_size=10, hidden_size=20, num_layers=2, batch_first=True)

gru = nn.GRU(input_size=10, hidden_size=20, num_layers=2, batch_first=True)


input_tensor = torch.randn(batch_size, 10)

# RNNCell 示例
rnn_cell = nn.RNNCell(input_size=10, hidden_size=20)
hidden = torch.zeros(batch_size, 20)
output = rnn_cell(input_tensor, hidden)



# LSTMCell 示例
lstm_cell = nn.LSTMCell(input_size=10, hidden_size=20)
hidden, cell = (torch.zeros(batch_size, 20), torch.zeros(batch_size, 20))
output = lstm_cell(input_tensor, (hidden, cell))

# GRUCell 示例
gru_cell = nn.GRUCell(input_size=10, hidden_size=20)
hidden = torch.zeros(batch_size, 20)
output = gru_cell(input_tensor, hidden)



# Transformers 

In [72]:
import torch
import torch.nn as nn

# 超参数
d_model = 512  # 模型维度
nhead = 8      # 注意力头数
num_encoder_layers = 6  # 编码器层数
num_decoder_layers = 6  # 解码器层数
dim_feedforward = 2048   # 前馈网络的维度
dropout = 0.1            # dropout率

# 构建Transformer模型
transformer = nn.Transformer(d_model=d_model, nhead=nhead,
                              num_encoder_layers=num_encoder_layers,
                              num_decoder_layers=num_decoder_layers,
                              dim_feedforward=dim_feedforward,
                              dropout=dropout)

# 输入张量（假设是批次大小为10，序列长度为20，特征维度为512）
src = torch.randn(20, 10, d_model)  # 源序列
tgt = torch.randn(20, 10, d_model)  # 目标序列

# 前向传播
output = transformer(src, tgt)

# 输出维度： (目标序列长度, 批次大小, 特征维度)
print(output.shape)  # 应该输出torch.Size([20, 10, 512])


c:\Users\epiph\.conda\envs\torchproject\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


torch.Size([20, 10, 512])


# Linear

In [1]:
# nn.Identity
import torch
import torch.nn as nn

identity_layer = nn.Identity()
input_data = torch.randn(5, 3)
output = identity_layer(input_data)

print(output.shape)  # 输出: torch.Size([5, 3])，与输入相同

torch.Size([5, 3])


In [2]:
# nn.Linear
linear_layer = nn.Linear(in_features=3, out_features=2)
input_data = torch.randn(5, 3)
output = linear_layer(input_data)

print(output.shape)  # 输出: torch.Size([5, 2])

torch.Size([5, 2])


In [3]:
# nn.Bilinear
bilinear_layer = nn.Bilinear(in1_features=4, in2_features=5, out_features=2)
input1 = torch.randn(3, 4)  # 第一个输入，大小为 (batch_size, in1_features)
input2 = torch.randn(3, 5)  # 第二个输入，大小为 (batch_size, in2_features)
output = bilinear_layer(input1, input2)

print(output.shape)  # 输出: torch.Size([3, 2])

torch.Size([3, 2])


In [4]:
# nn.LazyLinear 是 nn.Linear 的延迟版本，in_features 会在第一次前向传播时推断出来，这对于构建动态网络结构非常有用。
lazy_linear_layer = nn.LazyLinear(out_features=2)  # 不需要指定 in_features
input_data = torch.randn(3, 4)  # 输入数据
output = lazy_linear_layer(input_data)

print(output.shape)  # 输出: torch.Size([3, 2])

torch.Size([3, 2])


c:\Users\epiph\.conda\envs\torchproject\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


# DropOut Layers
Dropout对整个通道进行置零的操作，如果是Dropout，则相当于每一个通道都是一个0维的数，就是将常数置为0，后面2d和3d都是将整个通道置为0，包括里面的所有维度。

In [6]:
# nn.Dropout
import torch
import torch.nn as nn

dropout_layer = nn.Dropout(p=0.5)
input_data = torch.randn(3, 4)
output = dropout_layer(input_data)

print(output)  # 部分元素将被置为0

tensor([[ 1.5829,  1.5258,  3.7213, -2.9059],
        [-0.0000,  0.0000,  2.1286, -0.2770],
        [-3.7058, -1.2279,  0.4626, -0.0000]])


In [7]:
# nn.Dropout1d
dropout1d_layer = nn.Dropout1d(p=0.5)
input_data = torch.randn(3, 4, 10)  # 3 个样本，4 个通道，10 个特征
output = dropout1d_layer(input_data)

print(output)  # 整个通道将被置为 0

tensor([[[-0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000,
          -0.0000, -0.0000,  0.0000],
         [-3.2586, -0.0196,  3.3419,  1.0078,  0.1490,  0.6181, -1.5868,
          -0.0281, -0.4238,  1.6973],
         [-0.6080,  0.3559, -1.3751, -1.5537, -3.0022, -2.9056, -0.9778,
           1.5977,  0.7983,  2.3488],
         [-0.0000,  0.0000, -0.0000,  0.0000,  0.0000, -0.0000,  0.0000,
          -0.0000,  0.0000, -0.0000]],

        [[ 1.0811, -2.7068,  0.2929, -1.0467, -2.0723, -0.5709, -0.2004,
          -2.5476, -2.5667, -0.0902],
         [-1.1177,  2.1965, -2.8573,  2.2241,  2.6370,  2.4197,  1.7120,
          -0.7738,  2.0432, -2.0385],
         [ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000, -0.0000,  0.0000,
           0.0000, -0.0000, -0.0000],
         [ 0.0000, -0.0000,  0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
          -0.0000,  0.0000, -0.0000]],

        [[-1.0414, -0.3292, -1.6101, -2.5687,  3.1574, -1.1343,  0.9455,
           0.4257, -1.7143,  0.4190

In [11]:
# nn.Dropout2d
dropout2d_layer = nn.Dropout2d(p=0.5)
input_data = torch.randn(3, 4, 10, 10)  # 3 个样本，4 个通道，10x10 大小的特征图
output = dropout2d_layer(input_data)

print(output)  # 整个通道将被置为 0

tensor([[[[-0.0000,  0.0000,  0.0000,  ..., -0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
          ...,
          [-0.0000, -0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000,  ..., -0.0000, -0.0000, -0.0000]],

         [[ 0.3693,  2.7405, -2.1436,  ..., -2.3167, -0.9858, -0.2188],
          [-0.7458,  0.6100, -3.6795,  ..., -0.4737, -3.3482,  0.8720],
          [ 2.4779,  2.2467,  2.4052,  ..., -0.1706,  0.5075, -0.2248],
          ...,
          [ 1.3884,  3.4897, -1.5378,  ...,  0.7729, -1.1729, -0.6638],
          [-0.7910, -1.3153,  1.1112,  ...,  0.6105,  0.1698,  0.4469],
          [-0.1943, -1.8806,  2.4401,  ...,  1.1114, -1.8727, -1.7596]],

         [[-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000,  

In [24]:
# nn.Dropout3d
dropout3d_layer = nn.Dropout3d(p=0.5)
input_data = torch.randn(3, 4, 10, 10, 10)  # 3 个样本，4 个通道，10x10x10 大小的特征图
output = dropout3d_layer(input_data)

print(output)  # 整个通道将被置为 0


tensor([[[[[-0.0000e+00, -0.0000e+00,  0.0000e+00,  ..., -0.0000e+00,
            -0.0000e+00, -0.0000e+00],
           [ 0.0000e+00,  0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
            -0.0000e+00,  0.0000e+00],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
             0.0000e+00, -0.0000e+00],
           ...,
           [-0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
            -0.0000e+00,  0.0000e+00],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [-0.0000e+00,  0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
            -0.0000e+00, -0.0000e+00]],

          [[ 0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 0.0000e+00, -0.0000e+00, -0.0000e+00,  ...,  0.0000e+00,
            -0.0000e+00, -0.0000e+00],
 

In [27]:
# nn.AlphaDropout 是一种特殊的 Dropout，主要用于 SELU 激活函数，以保证均值和方差不变。
alpha_dropout_layer = nn.AlphaDropout(p=0.5)
input_data = torch.randn(3, 4)
output = alpha_dropout_layer(input_data)

print(output)  # 一部分输入会被随机置为 0

tensor([[-0.7792,  0.7763,  0.6064, -0.7792],
        [-0.7792,  0.8955, -0.7792,  0.7126],
        [-0.7792, -0.7792,  1.1647, -0.7792]])


In [28]:
# nn.FeatureAlphaDropout 类似于 AlphaDropout，但它会随机将整个特征通道置为 0。
feature_alpha_dropout_layer = nn.FeatureAlphaDropout(p=0.5)
input_data = torch.randn(3, 4, 10)  # 3 个样本，4 个通道，10 个特征
output = feature_alpha_dropout_layer(input_data)

print(output)  # 整个通道将被随机置为 0


tensor([[[-0.7792, -0.7792, -0.7792, -0.7792, -0.7792, -0.7792, -0.7792,
          -0.7792, -0.7792, -0.7792],
         [-0.7792, -0.7792, -0.7792, -0.7792, -0.7792, -0.7792, -0.7792,
          -0.7792, -0.7792, -0.7792],
         [ 0.7312,  0.7215,  1.0061, -0.1258,  0.8800,  0.0375,  1.2197,
           1.2128, -0.2303,  0.1198],
         [ 1.8267,  0.8541, -0.2851,  0.2515,  0.1064,  1.4023,  0.5962,
          -0.0202, -0.1699,  0.2069]],

        [[ 0.4767,  1.3892,  3.1136, -0.3216,  2.1921,  0.8499,  1.0830,
           1.0773,  1.6652,  2.0447],
         [ 0.7447,  0.0246,  1.9605,  1.3564,  0.8277,  1.6889,  0.0721,
           1.3913,  2.3715,  1.4535],
         [-0.7792, -0.7792, -0.7792, -0.7792, -0.7792, -0.7792, -0.7792,
          -0.7792, -0.7792, -0.7792],
         [ 0.1083,  2.2539,  1.3623,  1.3676,  1.2805,  1.0769, -0.5790,
           1.8392,  0.9436,  0.6383]],

        [[ 1.1084,  1.2945,  1.9404,  0.6845,  2.3187, -1.2364,  0.5870,
          -0.6565,  1.5612,  0.3740

# Sparse Layers Embedding

In [32]:
import torch
import torch.nn as nn

# 创建一个包含 10 个单词，每个单词嵌入为 3 维向量的 Embedding 层
embedding = nn.Embedding(num_embeddings=10, embedding_dim=3)

# 输入是单词的索引（这里假设词典中的索引是 [0, 1, 2, ...]）
input_indices = torch.tensor([1, 2, 6, 8])

# 通过 embedding 查找对应的嵌入向量
output = embedding(input_indices)

print(output)  # 输出 [4, 3] 的嵌入向量


tensor([[-0.6502,  2.3064,  0.1625],
        [ 0.1200,  1.4873, -0.9191],
        [ 0.0973, -1.4780, -0.3336],
        [-0.0181, -0.9556,  0.3699]], grad_fn=<EmbeddingBackward0>)


In [37]:
# 创建一个包含 10 个单词，每个单词嵌入为 3 维向量的 EmbeddingBag 层
embedding_bag = nn.EmbeddingBag(num_embeddings=10, embedding_dim=3, mode='mean') # 或 'sum' 或 'max'

# 输入是一个单词索引的 batch（多个 bag），并且每个 bag 的长度是可变的
input_indices = torch.tensor([1, 2, 5, 8])
offsets = torch.tensor([0, 2])  # 表示两个 bag，第一个 bag 是 [1, 2]，第二个 bag 是 [5, 8]
# offsets 表示每个 bag 的起始位置，即input_indices[0]和input_indices[2]

# 通过 embedding_bag 查找对应的嵌入向量，并对每个 bag 内的向量求平均
output = embedding_bag(input_indices, offsets)

print(output)  # 输出 [2, 3] 的 bag 平均嵌入


tensor([[-0.6909, -0.1191,  0.7472],
        [ 0.5338, -1.1827, -0.7355]], grad_fn=<EmbeddingBagBackward0>)


# Similarity

In [38]:
# nn.CosineSimilarity
import torch
import torch.nn as nn

# 初始化两个向量（这里假设有3个维度）
x1 = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])  # Shape: (2, 3)
x2 = torch.tensor([[1.0, 2.0, 3.0], [7.0, 8.0, 9.0]])  # Shape: (2, 3)

# 初始化 nn.CosineSimilarity，指定计算相似度的维度为最后一维
cos = nn.CosineSimilarity(dim=1)

# 计算两个向量之间的余弦相似度
similarity = cos(x1, x2)

print("余弦相似度: ", similarity)


余弦相似度:  tensor([1.0000, 0.9982])


In [39]:
import torch
import torch.nn as nn

# 初始化两个向量，每个包含两个样本，每个样本有三个特征
x1 = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])  # Shape: (2, 3)
x2 = torch.tensor([[1.0, 2.0, 3.0], [7.0, 8.0, 9.0]])  # Shape: (2, 3)

# 初始化 nn.PairwiseDistance 实例
pairwise_dist = nn.PairwiseDistance(p=2)  # p=2 表示欧氏距离

# 计算每对向量之间的距离
distance = pairwise_dist(x1, x2)

print("成对距离: ", distance)


成对距离:  tensor([1.7321e-06, 5.1962e+00])


### 1. **nn.L1Loss**
   - **用途**：L1 损失函数通常用于回归任务中，用来计算输入和目标值之间的**平均绝对误差**（MAE）。
   - **特点**：对**离群点**比较敏感，因为它对每个误差的绝对值进行平均处理，因此适合需要对误差较小的任务更敏感的场景。

### 2. **nn.MSELoss**
   - **用途**：常用于回归问题中，计算输入和目标之间的**均方误差**。
   - **特点**：更关注大的误差，惩罚较大的误差，适合场景中较大的误差需要额外处理的情况。

### 3. **nn.CrossEntropyLoss**
   - **用途**：用于**分类问题**，特别是多类分类问题。它计算输入的**logits**和目标类别的交叉熵损失。
   - **特点**：将`nn.LogSoftmax`和`nn.NLLLoss`结合在一起，适合用于需要概率输出的分类任务。

### 4. **nn.CTCLoss**
   - **用途**：用于**序列建模任务**，如语音识别，OCR等。它可以处理不对齐的输入序列和目标序列。
   - **特点**：支持**可变长度**的输出序列，对目标序列进行无对齐计算。

### 5. **nn.NLLLoss**
   - **用途**：用于分类问题，输入是对数概率（log-probabilities），用于测量负对数似然损失。
   - **特点**：通常与`nn.LogSoftmax`结合使用，适用于**概率估计**和分类任务。

### 6. **nn.PoissonNLLLoss**
   - **用途**：用于**计数预测问题**，其中目标服从**泊松分布**。
   - **特点**：适用于目标是非负整数并且符合泊松分布的任务，如事件计数问题。

### 7. **nn.GaussianNLLLoss**
   - **用途**：用于**回归任务**，假设目标值服从**高斯分布**。
   - **特点**：适用于目标具有均值和方差，并且符合高斯分布的情况。

### 8. **nn.KLDivLoss**
   - **用途**：用于计算两个概率分布之间的**Kullback-Leibler (KL) 散度**。
   - **特点**：通常用于训练变分自编码器（VAE）等场景中，用于测量两个分布的差异。

### 9. **nn.BCELoss**
   - **用途**：用于**二分类问题**，测量输入和目标的二元交叉熵。
   - **特点**：输出值应是概率（通常通过`Sigmoid`函数获得），适用于需要二分类的任务。

### 10. **nn.BCEWithLogitsLoss**
   - **用途**：结合了`Sigmoid`和`BCELoss`，用于二分类问题，避免了数值不稳定问题。
   - **特点**：比直接使用`BCELoss`更加稳定高效，因为将Sigmoid与损失计算合并为一个步骤。

### 11. **nn.MarginRankingLoss**
   - **用途**：用于**排序任务**，比较两个输入向量的大小差异，要求标签为1或-1。
   - **特点**：适用于信息检索、推荐系统等排序问题。

### 12. **nn.HingeEmbeddingLoss**
   - **用途**：用于**相似度学习**，用于嵌入向量的衡量，目标标签为1或-1。
   - **特点**：适合分类或度量学习中的正负样本对比任务。

### 13. **nn.MultiLabelMarginLoss**
   - **用途**：用于**多标签分类**任务，优化多类多标签的 hinge 损失。
   - **特点**：适用于需要处理多个目标标签的场景，如图像标注任务。

### 14. **nn.HuberLoss**
   - **用途**：用于回归任务，结合了`L1`和`L2`损失的优点。
   - **特点**：当误差较小时使用`L2`，误差较大时使用`L1`，适合处理**异常值**和**稳健回归**。

### 15. **nn.SmoothL1Loss**
   - **用途**：常用于目标检测等任务，损失在小误差时为平方项，大误差时为线性项。
   - **特点**：相比`MSELoss`更平滑和鲁棒，适用于回归问题中需要处理**异常值**的场景。

### 16. **nn.SoftMarginLoss**
   - **用途**：用于二分类任务，使用逻辑回归损失来优化。
   - **特点**：目标标签为1或-1，适用于二分类问题。

### 17. **nn.MultiLabelSoftMarginLoss**
   - **用途**：用于**多标签分类**，优化基于最大熵的多标签 one-vs-all 损失。
   - **特点**：适用于多标签分类任务，如文本分类。

### 18. **nn.CosineEmbeddingLoss**
   - **用途**：用于**相似度学习**，根据两个输入向量的余弦相似度衡量它们的相似性。
   - **特点**：在度量学习和相似度任务中常见，用于如人脸识别等。

### 19. **nn.MultiMarginLoss**
   - **用途**：用于**多类分类**任务，基于 hinge 损失。
   - **特点**：适用于需要分类多个类别的场景，通常与SVM类似。

### 20. **nn.TripletMarginLoss**
   - **用途**：用于**度量学习**中的三元组损失，衡量锚点、正样本和负样本之间的相对距离。
   - **特点**：常用于人脸识别和推荐系统中的相似度度量。

### 21. **nn.TripletMarginWithDistanceLoss**
   - **用途**：扩展三元组损失，允许使用不同的**距离函数**来计算损失。
   - **特点**：适用于自定义距离度量的三元组学习任务。

# vision layers

### 1. **nn.PixelShuffle**
   - **用途**：将一个低分辨率的特征图重新排列成更高分辨率的特征图，通常用于**超分辨率**任务。
   - **特点**：输入是一个包含多个通道的特征图，经过重排列后通道数减少，空间维度增加。常用于提高图像分辨率时的最后一步。

### 2. **nn.PixelUnshuffle**
   - **用途**：执行与`PixelShuffle`相反的操作，将高分辨率图像重新排列成低分辨率的特征图。
   - **特点**：减少空间维度，增加通道数，适用于需要降维或特征提取的场景。

### 3. **nn.Upsample**
   - **用途**：对输入的1D、2D或3D多通道数据进行上采样，增加分辨率。
   - **特点**：可以指定插值方式（如`nearest`或`linear`），适用于**图像增强**或特征图的放大。

### 4. **nn.UpsamplingNearest2d**
   - **用途**：对2D输入信号（如图像）使用最近邻插值方法进行上采样。
   - **特点**：插值方式简单，计算效率高，但插值结果不如其他方法平滑，适用于需要快速上采样的场景。

### 5. **nn.UpsamplingBilinear2d**
   - **用途**：对2D输入信号使用双线性插值方法进行上采样。
   - **特点**：相比最近邻插值，双线性插值可以生成更平滑的图像，适用于对图像质量要求较高的上采样任务。

In [40]:
import torch
import torch.nn as nn

# 输入张量，假设是一个2D图像，形状为(batch_size, channels, height, width)
input_tensor = torch.randn(1, 1, 4, 4)  # 1个样本，1个通道，4x4的图像

# 1. nn.PixelShuffle这里将输入的4通道(2x2)张量上采样为1通道(4x4)，这里将特征图拓展了，但是将通道数减少了
def pixel_shuffle_example(input_tensor, upscale_factor):
    # 在这里，我们需要调整通道数以使用PixelShuffle
    # 比如，将输入的通道数调整为 (upscale_factor^2) * c
    # 在这个例子中，假设我们要将 1 个通道转变为 4 个通道 (2x2)
    input_tensor = input_tensor.view(1, 4, 2, 2)  # 1个样本，4个通道，2x2
    pixel_shuffle = nn.PixelShuffle(upscale_factor)
    output_pixel_shuffle = pixel_shuffle(input_tensor)
    return output_pixel_shuffle

print("PixelShuffle Output:\n", pixel_shuffle_example(input_tensor, upscale_factor=2))


# 2. nn.PixelUnshuffle将输入的张量恢复到低分辨率，输出的是多个通道，也就是将特征图缩小，对应使通道数增加
def pixel_unshuffle_example(input_tensor, upscale_factor):
    pixel_unshuffle = nn.PixelUnshuffle(upscale_factor)
    # 假设我们已经有了一个输出形状为 (1, 4, 4, 4)，即输入形状为 (1, 1, 4, 4)
    input_tensor = input_tensor.view(1, 1, 4, 4)  # 1个样本，1个通道，4x4
    output_pixel_unshuffle = pixel_unshuffle(input_tensor)
    return output_pixel_unshuffle

print("PixelUnshuffle Output:\n", pixel_unshuffle_example(input_tensor, upscale_factor=2))


# 3. nn.Upsample直接对输入张量上采样到8x8的分辨率。
def upsample_example(input_tensor):
    upsample = nn.Upsample(size=(8, 8), mode='bilinear', align_corners=False)
    output_upsample = upsample(input_tensor)
    return output_upsample

print("Upsample Output:\n", upsample_example(input_tensor))


# 4. nn.UpsamplingNearest2d使用最近邻插值方法将张量上采样到8x8。
def upsampling_nearest_example(input_tensor):
    upsampling_nearest = nn.UpsamplingNearest2d(size=(8, 8))
    output_upsampling_nearest = upsampling_nearest(input_tensor)
    return output_upsampling_nearest

print("UpsamplingNearest2d Output:\n", upsampling_nearest_example(input_tensor))


# 5. nn.UpsamplingBilinear2d使用双线性插值方法将张量上采样到8x8。
def upsampling_bilinear_example(input_tensor):
    upsampling_bilinear = nn.UpsamplingBilinear2d(size=(8, 8))
    output_upsampling_bilinear = upsampling_bilinear(input_tensor)
    return output_upsampling_bilinear

print("UpsamplingBilinear2d Output:\n", upsampling_bilinear_example(input_tensor))


PixelShuffle Output:
 tensor([[[[ 0.9690,  0.2592,  0.5729,  0.4074],
          [ 0.0354, -0.3096,  0.1912,  0.0859],
          [-0.0361, -0.6968,  1.9412,  0.4971],
          [-0.9405,  0.0198,  0.2116,  0.6856]]]])
PixelUnshuffle Output:
 tensor([[[[ 0.9690, -0.0361],
          [ 0.0354, -0.9405]],

         [[ 0.5729,  1.9412],
          [ 0.1912,  0.2116]],

         [[ 0.2592, -0.6968],
          [-0.3096,  0.0198]],

         [[ 0.4074,  0.4971],
          [ 0.0859,  0.6856]]]])
Upsample Output:
 tensor([[[[ 0.9690,  0.8700,  0.6719,  0.4207,  0.1162,  0.4583,  1.4469,
            1.9412],
          [ 0.7915,  0.7265,  0.5965,  0.3484, -0.0180,  0.2441,  1.1348,
            1.5801],
          [ 0.4367,  0.4397,  0.4458,  0.2037, -0.2865, -0.1842,  0.5107,
            0.8581],
          [ 0.2033,  0.2408,  0.3158,  0.0756, -0.4799, -0.4619,  0.1299,
            0.4257],
          [ 0.0914,  0.1299,  0.2068, -0.0359, -0.5983, -0.5889, -0.0076,
            0.2830],
          [-0.050

# Shuffle Layers

In [41]:
import torch
import torch.nn as nn

# 创建一个输入张量，假设形状为 (batch_size, channels, height, width)
input_tensor = torch.randn(1, 4, 4, 4)  # 1个样本，4个通道，4x4的图像

# 定义 ChannelShuffle 函数
def channel_shuffle_example(input_tensor, groups):
    channel_shuffle = nn.ChannelShuffle(groups)
    output_channel_shuffle = channel_shuffle(input_tensor)
    return output_channel_shuffle

# 使用 2 组来进行通道混洗
groups = 2
output = channel_shuffle_example(input_tensor, groups)

print("Input Tensor Shape: ", input_tensor.shape)
print("Output Tensor Shape: ", output.shape)


Input Tensor Shape:  torch.Size([1, 4, 4, 4])
Output Tensor Shape:  torch.Size([1, 4, 4, 4])
